Included for reproducibility. Converts synthetic data into aesthetics regression datasets.

In [2]:
from bikebench.resource_utils import datasets_path

In [3]:
CTGAN_dir = datasets_path("Synthetic_Extended_Data/CTGAN")
uniform_dir = datasets_path("Synthetic_Extended_Data/uniform")


In [9]:
#loop over each directory/designs and load 1-200.csv
import pandas as pd
from tqdm import tqdm

designs = []
embeddings = []
for dir in [CTGAN_dir, uniform_dir]:
    for design in tqdm(range(0, 200)):
        try:
            file_path = f"{dir}/designs/{design}.csv"
            des_df = pd.read_csv(file_path, index_col=0)
            assert des_df.shape == (4096, 83) 

            file_path = f"{dir}/embeddings/{design}.csv"
            emb_df = pd.read_csv(file_path, index_col=0)

            designs.append(des_df)
            embeddings.append(emb_df)
        except Exception as e:
            print(f"Error loading design {design} in {dir}: {e}")
designs_df = pd.concat(designs, ignore_index=False)
embeddings_df = pd.concat(embeddings, ignore_index=False)


100%|██████████| 200/200 [00:34<00:00,  5.82it/s]


In [10]:
print(designs_df.shape, embeddings_df.shape)

(1638400, 83) (1638399, 512)


In rare cases, the rendering process can fail. This seems to have happened for one design in the 1.6M. We will drop it. 

In [11]:
#drop any indexes that are in embeddings but not in designs and vice versa
common_indexes = designs_df.index.intersection(embeddings_df.index)
designs_df = designs_df.loc[common_indexes]
embeddings_df = embeddings_df.loc[common_indexes]

In [12]:
print(designs_df.shape, embeddings_df.shape)

(1638399, 83) (1638399, 512)


Save as csvs (additionally save a float16 numpy version for convenience)

In [13]:
import numpy as np
designs_df.to_csv(datasets_path("Predictive_Modeling_Datasets/aesthetics_X_train.csv"))
# embeddings_np = embeddings_df.to_numpy().astype(np.float16)
# np.save(datasets_path("Predictive_Modeling_Datasets/aesthetics_Y_train.npy"), embeddings_np)
# embeddings_df.to_csv(datasets_path("Predictive_Modeling_Datasets/aesthetics_Y_train.csv"))
